In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta     

In [2]:
NGR=pd.read_csv('playerstats.csv')

In [3]:
# Deletando a última linha do DataFrame
NGR = NGR.drop(NGR.index[-1])

In [4]:
NGR["First deposit"] =pd.to_datetime(NGR["First deposit"], errors = 'coerce')

In [5]:
NGR["Advertiser"].unique()

array(['3snet18432', 'mobytize', 'dmidd47', 'intimads', 'marketin',
       'invivoads', 'AlfaleadsCPA', 'defaultgalera.bet', 'gohike', 'bm22',
       'IdentiBRDisplay', 'ingoogleads', 'ally0034', 'ally0004',
       'intaboola', 'ally0031', 'ally0011', 'growthleads', 'ally0032',
       'allyinf0011', 'ally0028', 'leadbitolga', 'vmmeightroom',
       'LiqesGA_Mines', 'rubju300', 'knownmedia', 'premierleads',
       'fernando-mktesportivo', 'nsnwab', 'leadstarbr', 'pafeit',
       'seoglr', 'ally0016', 'ally0030', 'supera088', 'Affunlim',
       'Gn8282', 'jafe1000', 'ally0002', 'thago', 'jarbasbezerra05',
       'ally0021', '3snet1010', 'bc22ppc2', 'PremierLeads2',
       'EonF.Galera', 'ally0039', 'ally0012', 'breakmedia', 'samir',
       'inpropellerads', 'jompa', 'rspcom', 'crmbonus', 'Ineditoraglobo',
       'bwise_influencers', 'glaununes', 'liqesGA', 'equadratBR',
       'marcus.igoalcorp', 'oddsscannerbr', 'g1odds', 'acroud',
       '3snet666', 'liqes', 'scalemedia', 'ally0005', '

In [6]:
NGR["Calculated_NGR"] = NGR['Casino Real Bets'] - NGR['Casino Real Wins'] -NGR['Redeemed bonus initial amounts'] - NGR['Redeemed bonus pending winnings']

In [7]:
NGR['First deposit date abreviated'] =NGR["First deposit"].dt.strftime('%m/%Y')

In [8]:
NGR.columns.values

array(['Username', 'Country', 'Advertiser', 'Signed up', 'First deposit',
       'Last deposit date', 'Wants mail', 'Deposits', 'Deposit count',
       'Average deposit', 'Approved Withdrawals',
       'Approved Withdrawal count', 'Casino NGR', 'Casino bonuses',
       'Casino bonus count', 'Redeemed bonus initial amounts',
       'Redeemed bonus pending winnings', 'Poker bonus reverses',
       'Casino progressive share', 'Casino progressive wins',
       'Casino Bets', 'Casino Wins', 'Casino Real Bets',
       'Casino Real Wins', 'Casino game count', 'Casino bonuses removed',
       'Frozen', 'Sportsbook wins', 'Sportsbook bets',
       'Sportsbook gross income', 'Calculated_NGR',
       'First deposit date abreviated'], dtype=object)

In [9]:
output = NGR.groupby('First deposit date abreviated')[["Calculated_NGR","Deposits","Deposit count"]].sum().reset_index()

In [10]:
output.loc[len(output)] = ["Total",output["Calculated_NGR"].sum(),output["Deposits"].sum(),output["Deposit count"].sum()]



In [11]:
import requests
# Conversão para DataFrame
df = pd.DataFrame(output)

# Renomeando colunas
df.rename(columns={
    "First deposit date abreviated": "Mês",
    "Calculated_NGR": "NGR",
    "Deposits": "Depósitos"
}, inplace=True)

# Fazendo a requisição para a API de câmbio (exemplo com Exchange Rates API)
api_key = "YOUR_API_KEY"  # Substitua pela sua chave de API
url = f"https://api.exchangerate-api.com/v4/latest/BRL"
response = requests.get(url)
data = response.json()

# Obter a taxa de conversão de BRL para USD
exchange_rate = data["rates"]["USD"]

# Adicionando colunas em USD
df["NGR_USD"] = df["NGR"] * exchange_rate
df["Depósitos_USD"] = df["Depósitos"] * exchange_rate

# Formatando os valores em R$ e em US$
df["NGR"] = df["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["Depósitos"] = df["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["NGR_USD"] = df["NGR_USD"].apply(lambda x: f'${x:,.2f}')
df["Depósitos_USD"] = df["Depósitos_USD"].apply(lambda x: f'${x:,.2f}')

df

,Mês,NGR,Depósitos,Deposit count,NGR_USD,Depósitos_USD
0,01/2022,"R$458,32","R$1.076,00",15,$77.00,$180.77
1,01/2023,"R$12.504,53","R$21.350,32",395,"$2,100.76","$3,586.85"
2,01/2024,"R$18.265,76","R$40.653,34",607,"$3,068.65","$6,829.76"
3,02/2022,"R$-1.261,37","R$4.780,98",99,$-211.91,$803.20
4,02/2023,"R$-16.043,26","R$28.325,08",352,"$-2,695.27","$4,758.61"
5,02/2024,"R$6.387,21","R$39.075,97",606,"$1,073.05","$6,564.76"
6,03/2022,"R$-11.264,58","R$25.619,08",196,"$-1,892.45","$4,304.01"
7,03/2023,"R$576,55","R$19.851,83",285,$96.86,"$3,335.11"
8,03/2024,"R$15.175,37","R$37.327,77",516,"$2,549.46","$6,271.07"
9,04/2022,"R$-2.640,39","R$29.262,93",361,$-443.59,"$4,916.17"


In [12]:
import pandas as pd
import requests
# Supondo que df já está criado conforme o exemplo dado

# Conversão para DataFrame
df = pd.DataFrame(output)

# Renomeando colunas
df.rename(columns={
    "First deposit date abreviated": "Mês",
    "Calculated_NGR": "NGR",
    "Deposits": "Depósitos"
}, inplace=True)

# Separando o ano e mês
df['Ano'] = df['Mês'].str.split('/').str[1]
df['Mês_num'] = df['Mês'].str.split('/').str[0]


# Fazendo a requisição para a API de câmbio (exemplo com Exchange Rates API)
api_key = "YOUR_API_KEY"  # Substitua pela sua chave de API
url = f"https://api.exchangerate-api.com/v4/latest/BRL"
response = requests.get(url)
data = response.json()

# Obter a taxa de conversão de BRL para USD
exchange_rate = data["rates"]["USD"]

# Adicionando colunas em USD
df["NGR_USD"] = df["NGR"] * exchange_rate
df["Depósitos_USD"] = df["Depósitos"] * exchange_rate

# Filtrando dados por ano (2022, 2023 e ano atual 2024)
df_2022 = df[df['Ano'] == '2022'].copy()
df_2023 = df[df['Ano'] == '2023'].copy()
df_ano_atual = df[df['Ano'] == '2024']

# Somando os valores por ano, incluindo conversão para USD
sum_2022 = {
    'Mês': '2022',
    'NGR': f"R${df_2022['NGR'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos': f"R${df_2022['Depósitos'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'NGR_USD': f"${df_2022['NGR_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos_USD': f"${df_2022['Depósitos_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Deposit count': df_2022['Deposit count'].sum()
}

sum_2023 = {
    'Mês': '2023',
    'NGR': f"R${df_2023['NGR'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos': f"R${df_2023['Depósitos'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'NGR_USD': f"${df_2023['NGR_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos_USD': f"${df_2023['Depósitos_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Deposit count': df_2023['Deposit count'].sum()
}

# Calculando o total geral
total_geral = {
    'Mês': 'Total',
    'NGR': f"R${pd.concat([df_2022['NGR'], df_2023['NGR'], df_ano_atual['NGR']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos': f"R${pd.concat([df_2022['Depósitos'], df_2023['Depósitos'], df_ano_atual['Depósitos']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'NGR_USD': f"${pd.concat([df_2022['NGR_USD'], df_2023['NGR_USD'], df_ano_atual['NGR_USD']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos_USD': f"${pd.concat([df_2022['Depósitos_USD'], df_2023['Depósitos_USD'], df_ano_atual['Depósitos_USD']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Deposit count': df_2022['Deposit count'].sum() + df_2023['Deposit count'].sum() + df_ano_atual['Deposit count'].sum()
}

# Removendo o ano de cada mês do ano atual (mantendo o formato "MM/AAAA")
df_ano_atual['Mês'] = df_ano_atual['Mês_num'] + '/' + df_ano_atual['Ano']

# Concatenando os resultados e adicionando colunas de conversão para USD
df_final = pd.concat([pd.DataFrame([sum_2022]), pd.DataFrame([sum_2023]), df_ano_atual, pd.DataFrame([total_geral])], ignore_index=True)

# Adicionando a coluna de ordem
df_final['Ordem'] = range(1, len(df_final) + 1)

# Reordenando as colunas para colocar "Ordem" como a primeira coluna
df_final = df_final[['Ordem', 'Mês', 'NGR', 'Depósitos', 'NGR_USD', 'Depósitos_USD', 'Deposit count']]

# Exibindo o resultado final
df_final

C:\Users\Paulo\AppData\Local\Temp\ipykernel_17328\3534059534.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ano_atual['Mês'] = df_ano_atual['Mês_num'] + '/' + df_ano_atual['Ano']


,Ordem,Mês,NGR,Depósitos,NGR_USD,Depósitos_USD,Deposit count
0,1,2022,"R$85.223,09","R$379.810,58","$14.317,48","$63.808,18",4877
1,2,2023,"R$41.274,35","R$265.414,54","$6.934,09","$44.589,64",4354
2,3,01/2024,18265.76,40653.34,3068.64768,6829.76112,607
3,4,02/2024,6387.21,39075.97,1073.05128,6564.76296,606
4,5,03/2024,15175.37,37327.77,2549.46216,6271.06536,516
5,6,04/2024,18542.76,57923.16,3115.18368,9731.09088,739
6,7,05/2024,16414.24,58812.02,2757.59232,9880.41936,1048
7,8,06/2024,-1274.19,74486.97,-214.06392,12513.81096,1220
8,9,07/2024,11492.22,77863.63,1930.69296,13081.08984,1194
9,10,08/2024,16117.93,88502.72,2707.81224,14868.45696,1340


In [13]:
# Aplicando a formatação de moeda para o ano atual
df_ano_atual["NGR"] = df_ano_atual["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df_ano_atual["Depósitos"] = df_ano_atual["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df_ano_atual["NGR_USD"] = df_ano_atual["NGR_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df_ano_atual["Depósitos_USD"] = df_ano_atual["Depósitos_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))

# Concatenando os resultados e adicionando colunas de conversão para USD
df_final = pd.concat([pd.DataFrame([sum_2022]), pd.DataFrame([sum_2023]), df_ano_atual, pd.DataFrame([total_geral])], ignore_index=True)

# Adicionando a coluna de ordem
df_final['Ordem'] = range(1, len(df_final) + 1)

# Reordenando as colunas para colocar "Ordem" como a primeira coluna
df_final = df_final[['Ordem', 'Mês', 'NGR', 'Depósitos', 'NGR_USD', 'Depósitos_USD', 'Deposit count']]

df_final['Data'] = "12/12/2024"


# Exibindo o resultado final
df_final


C:\Users\Paulo\AppData\Local\Temp\ipykernel_17328\3125865402.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ano_atual["NGR"] = df_ano_atual["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
C:\Users\Paulo\AppData\Local\Temp\ipykernel_17328\3125865402.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ano_atual["Depósitos"] = df_ano_atual["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
C:\Users\Paulo\AppDa

,Ordem,Mês,NGR,Depósitos,NGR_USD,Depósitos_USD,Deposit count,Data
0,1,2022,"R$85.223,09","R$379.810,58","$14.317,48","$63.808,18",4877,12/12/2024
1,2,2023,"R$41.274,35","R$265.414,54","$6.934,09","$44.589,64",4354,12/12/2024
2,3,01/2024,"R$18.265,76","R$40.653,34","$3.068,65","$6.829,76",607,12/12/2024
3,4,02/2024,"R$6.387,21","R$39.075,97","$1.073,05","$6.564,76",606,12/12/2024
4,5,03/2024,"R$15.175,37","R$37.327,77","$2.549,46","$6.271,07",516,12/12/2024
5,6,04/2024,"R$18.542,76","R$57.923,16","$3.115,18","$9.731,09",739,12/12/2024
6,7,05/2024,"R$16.414,24","R$58.812,02","$2.757,59","$9.880,42",1048,12/12/2024
7,8,06/2024,"R$-1.274,19","R$74.486,97","$-214,06","$12.513,81",1220,12/12/2024
8,9,07/2024,"R$11.492,22","R$77.863,63","$1.930,69","$13.081,09",1194,12/12/2024
9,10,08/2024,"R$16.117,93","R$88.502,72","$2.707,81","$14.868,46",1340,12/12/2024


In [14]:
import uuid

# Carregar o arquivo CSV existente
df_existing = pd.read_excel("daily_playerstats1.xlsx")

# Filtrar as colunas necessárias (incluindo 'ID')
df_existing = df_existing[["Mês", "NGR", "Depósitos", "Deposit count", "NGR_USD", "Depósitos_USD", "Data", "ID", "Ordem"]]

# Gerar um ID único para toda a nova tabela 'final_result'
unique_id = str(uuid.uuid4())

# Adicionar a mesma ID para todas as linhas da nova tabela
df_final['ID'] = unique_id

# Concatenar o DataFrame novo (final_result) ao existente
df_combined = pd.concat([df_existing, df_final], ignore_index=True)

# Remover duplicatas com base na coluna "Username" (ou outra coluna de sua escolha)
#df_combined = df_combined.drop_duplicates(subset=["Ordem"], keep="last")

# Salvar o DataFrame combinado e sem duplicatas no arquivo CSV
df_combined.to_excel("daily_playerstats1.xlsx",index=False)